In [ ]:
url = f'https://sfs-msc-pub-lq-05.navigator.dot.ga.gov/rtplive/GDOT-CCTV-0563/playlist.m3u8'
url

In [ ]:
from urllib.parse import urljoin
import urllib.request
import requests
import re
import urllib
from pathlib import Path
import subprocess
import time
import os
from os import path


while True:
    num = urljoin(url, '.')[-3:-1]
    Path(f'rawmedia/{num}/').mkdir(exist_ok=True, parents=True)
    Path(f'media/{num}/').mkdir(exist_ok=True, parents=True)

    file_ids = [f[f.rfind('_')+1:-3] for f in os.listdir(f'rawmedia/{num}') if path.isfile(path.join(f'rawmedia/{num}', f))]

    base_url = url[:url.rfind('/') + 1]
    playlist = requests.get(url).text
    file_name = re.findall(r'chunklist_w.*?\.m3u8', playlist)
    if (len(file_name) == 0):
        exit()
    file_name = file_name[-1]
    chunklist_url = base_url + file_name
    chunklist = requests.get(chunklist_url).text
    media_files = re.findall(r'media_w.*?\.ts', chunklist)
    media_concat = 'concat:'
    for media_file in media_files:
        if media_file[media_file.rfind('_')+1:-3] in file_ids:
            print('s', end='')
            continue
        print(f'downloading {media_file}')
        urllib.request.urlretrieve(base_url + media_file, f'rawmedia/{num}/{media_file}')
        subprocess.run(['ffmpeg', '-i', 
                        f'rawmedia/{num}/{media_file}', 
                        f'media/{num}/media{len(os.listdir(f'media/{num}'))}.mp4'])
    time.sleep(3)

